In [ ]:
from sklearn.datasets import load_breast_cancer
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_boosting_df = pd.read_csv('/content/drive/MyDrive/Constraint/Boosting/Boosting Data/Train_v3.csv')
# train_df = train_df.rename(columns={'model1_Class1.1': 'model2_Class1', 'model1_Class1.2': 'model3_Class1'})
val_boosting_df = pd.read_csv('/content/drive/MyDrive/Constraint/Boosting/Boosting Data/Validation_v3.csv')
# val_df = val_df.rename(columns={'model1_Class1.1': 'model2_Class1', 'model1_Class1.2': 'model3_Class1'})
test_df = pd.read_csv('/content/drive/MyDrive/Constraint/data/Constraint_Test.csv')
val_df = pd.read_csv('/content/drive/MyDrive/Constraint/data/Constraint_Val.csv')
test_boosting_df = pd.read_csv('/content/drive/MyDrive/Constraint/Boosting/Boosting Data/Test_v3.csv')


In [ ]:
train_boosting_df.shape, val_boosting_df.shape, test_boosting_df.shape

((6420, 12), (2140, 12), (2140, 11))

In [ ]:
ernie_val = pd.read_csv('/content/drive/MyDrive/Constraint/Boosting/Intermediate/ERNIE/val_ernie2.0_results.csv')
ernie_val = ernie_val.rename(columns={'Model4_class0': 'model6_Class0', 'Model4_class1': 'model6_Class1'})[["model6_Class0", "model6_Class1", "Label"]]
ernie_test = pd.read_csv('/content/drive/MyDrive/Constraint/Boosting/Intermediate/ERNIE/test_ernie2.0_results.csv')
ernie_test = ernie_test.rename(columns={'Model4_class0': 'model6_Class0', 'Model4_class1': 'model6_Class1'})[["model6_Class0", "model6_Class1"]]


In [ ]:
test_boosting_df.head(3)

,id,model1_Class0,model1_Class1,model2_Class0,model2_Class1,model3_Class0,model3_Class1,model4_Class0,model4_Class1,model5_Class0,model5_Class1
0,1,0.999851,0.000149,0.999063,0.000937,0.999967,0.000033,0.999963,0.000037,0.999984,0.000016
1,2,0.000422,0.999578,0.000902,0.999098,0.000045,0.999955,0.000112,0.999888,0.000067,0.999933
2,3,0.000401,0.999600,0.000954,0.999046,0.000033,0.999967,0.000170,0.999830,0.000081,0.999919


In [ ]:
val_labels = val_boosting_df['Label'].values
val_boosting_df = pd.concat([val_boosting_df.iloc[:, :-1], ernie_val[["model6_Class0", "model6_Class1"]]], 1)
val_boosting_df["Label"] = val_labels
val_boosting_df.head()


,id,model1_Class0,model1_Class1,model2_Class0,model2_Class1,model3_Class0,model3_Class1,model4_Class0,model4_Class1,model5_Class0,model5_Class1,model6_Class0,model6_Class1,Label
0,1,0.001269,0.998731,0.000938,0.999062,0.000179,0.999821,0.028954,0.971046,0.000074,0.999926,0.000250,0.999750,1
1,2,0.991674,0.008326,0.000884,0.999116,0.010019,0.989981,0.002718,0.997282,0.958628,0.041371,0.005722,0.994278,1
2,3,0.000385,0.999615,0.000958,0.999042,0.000084,0.999916,0.000053,0.999947,0.000048,0.999952,0.000249,0.999751,1
3,4,0.000416,0.999584,0.000936,0.999065,0.000109,0.999891,0.001268,0.998732,0.001172,0.998828,0.000251,0.999749,1
4,5,0.999851,0.000149,0.999052,0.000948,0.999784,0.000216,0.999951,0.000049,0.999972,0.000028,0.999373,0.000627,0


In [ ]:
test_boosting_df = pd.concat([test_boosting_df, ernie_test[["model6_Class0", "model6_Class1"]]], 1)
test_boosting_df.head()

,id,model1_Class0,model1_Class1,model2_Class0,model2_Class1,model3_Class0,model3_Class1,model4_Class0,model4_Class1,model5_Class0,model5_Class1,model6_Class0,model6_Class1
0,1,0.999851,0.000149,0.999063,0.000937,0.999967,0.000033,0.999963,0.000037,0.999984,0.000016,0.999371,0.000629
1,2,0.000422,0.999578,0.000902,0.999098,0.000045,0.999955,0.000112,0.999888,0.000067,0.999933,0.000257,0.999743
2,3,0.000401,0.999600,0.000954,0.999046,0.000033,0.999967,0.000170,0.999830,0.000081,0.999919,0.000247,0.999753
3,4,0.999858,0.000142,0.999082,0.000918,0.999967,0.000033,0.999956,0.000044,0.999984,0.000016,0.999376,0.000624
4,5,0.999852,0.000148,0.999076,0.000924,0.999953,0.000047,0.999912,0.000088,0.999984,0.000016,0.994812,0.005188


In [ ]:
ernie_val.shape, ernie_test.shape

((2140, 3), (2140, 2))

In [ ]:
test_boosting_df.shape

(2140, 13)

In [ ]:
test_boosting_df.head()

,id,model1_Class0,model1_Class1,model2_Class0,model2_Class1,model3_Class0,model3_Class1,model4_Class0,model4_Class1,model5_Class0,model5_Class1,model6_Class0,model6_Class1
0,1,0.999851,0.000149,0.999063,0.000937,0.999967,0.000033,0.999963,0.000037,0.999984,0.000016,0.999371,0.000629
1,2,0.000422,0.999578,0.000902,0.999098,0.000045,0.999955,0.000112,0.999888,0.000067,0.999933,0.000257,0.999743
2,3,0.000401,0.999600,0.000954,0.999046,0.000033,0.999967,0.000170,0.999830,0.000081,0.999919,0.000247,0.999753
3,4,0.999858,0.000142,0.999082,0.000918,0.999967,0.000033,0.999956,0.000044,0.999984,0.000016,0.999376,0.000624
4,5,0.999852,0.000148,0.999076,0.000924,0.999953,0.000047,0.999912,0.000088,0.999984,0.000016,0.994812,0.005188


In [ ]:
def soft_voting(row):
  # class0_val = (row['model1_Class0'] + row['model2_Class0'] + row['model3_Class0'] + row['model4_Class0'] + row['model5_Class0'] + row['model6_Class0'])/6
  # class1_val = (row['model1_Class1'] + row['model2_Class1'] + row['model3_Class1'] + row['model4_Class1'] + row['model5_Class1'] + row['model6_Class1'])/6
  class0_val = (row['model1_Class0'] + row['model2_Class0'] + row['model3_Class0'] + row['model5_Class0'] + row['model6_Class0'])/5
  class1_val = (row['model1_Class1'] + row['model2_Class1'] + row['model3_Class1'] + row['model5_Class1'] + row['model6_Class1'])/5
  return 0 if class0_val > class1_val else 1

val_boosting_df['Soft_Vote'] = val_boosting_df.apply(lambda x: soft_voting(x), 1)
test_boosting_df['Soft_Vote'] = test_boosting_df.apply(lambda x: soft_voting(x), 1)


In [ ]:
test_boosting_df.head(2)

,id,model1_Class0,model1_Class1,model2_Class0,model2_Class1,model3_Class0,model3_Class1,model4_Class0,model4_Class1,model5_Class0,model5_Class1,model6_Class0,model6_Class1,Soft_Vote
0,1,0.999851,0.000149,0.999063,0.000937,0.999967,0.000033,0.999963,0.000037,0.999984,0.000016,0.999371,0.000629,0
1,2,0.000422,0.999578,0.000902,0.999098,0.000045,0.999955,0.000112,0.999888,0.000067,0.999933,0.000257,0.999743,1


In [ ]:
def hard_voting(row):
  a = np.argmax(np.array([[row['model1_Class0'], row['model1_Class1']]]), 1)[0]
  b = np.argmax(np.array([[row['model2_Class0'], row['model2_Class1']]]), 1)[0]
  c = np.argmax(np.array([[row['model3_Class0'], row['model3_Class1']]]), 1)[0]
  d = np.argmax(np.array([[row['model4_Class0'], row['model4_Class1']]]), 1)[0]
  e = np.argmax(np.array([[row['model5_Class0'], row['model5_Class0']]]), 1)[0]
  f = np.argmax(np.array([[row['model6_Class0'], row['model6_Class0']]]), 1)[0]

  cnt_0 = 0
  for i in [a,b,c,d,e,f]:
    if i==0:
      cnt_0+=1
  
  if cnt_0>3:
    return 0
  elif cnt_0==3:
    class0_val = (row['model1_Class0'] + row['model2_Class0'] + row['model3_Class0'] + row['model4_Class0'] + row['model5_Class0'] + row['model6_Class0'])/6
    class1_val = (row['model1_Class1'] + row['model2_Class1'] + row['model3_Class1'] + row['model4_Class1'] + row['model5_Class1'] + row['model6_Class1'])/6
    return 0 if class0_val > class1_val else 1                                                                                                              
  else:
    return 1

val_boosting_df['Hard_Vote'] = val_boosting_df.apply(lambda x: hard_voting(x), 1)
test_boosting_df['Hard_Vote'] = test_boosting_df.apply(lambda x: hard_voting(x), 1)


In [ ]:
def label_id_to_label(row, col):
  return 'real' if row[col]==0 else 'fake'

val_boosting_df['label'] = val_boosting_df.apply(lambda x: label_id_to_label(x, 'Soft_Vote'), 1)
test_boosting_df['label'] = test_boosting_df.apply(lambda x: label_id_to_label(x, 'Soft_Vote'), 1)


In [ ]:
test_df = test_boosting_df[["id", "label"]]

In [ ]:
test_df.head()

,id,label
0,1,real
1,2,fake
2,3,fake
3,4,real
4,5,real


In [ ]:
# test_df.to_csv('/content/drive/MyDrive/Constraint/Submission_v4.csv', index=False)

In [ ]:
confusion_matrix(val_boosting_df['Label'].values, val_boosting_df['Soft_Vote'].values)

array([[1113,    7],
       [  27,  993]])

In [ ]:
accuracy_score(val_boosting_df['Label'].values, val_boosting_df['Soft_Vote'].values)

0.9841121495327103

In [ ]:
confusion_matrix(val_boosting_df['Label'].values, val_boosting_df['Hard_Vote'].values)

array([[1116,    4],
       [  48,  972]])

In [ ]:
accuracy_score(val_boosting_df['Label'].values, val_boosting_df['Hard_Vote'].values)

0.9757009345794393

In [ ]:
f1_score(val_boosting_df['Label'].values, val_boosting_df['Soft_Vote'].values)

0.9831683168316832

In [ ]:
def get_label(row):
  return 'real' if row["Label"]==0 else 'fake'

val_boosting_df["Actual_Label"] = val_boosting_df.apply(lambda x: get_label(x), 1)


In [ ]:
val_boosting_df.head()

,id,model1_Class0,model1_Class1,model2_Class0,model2_Class1,model3_Class0,model3_Class1,model4_Class0,model4_Class1,model5_Class0,model5_Class1,model6_Class0,model6_Class1,Label,Soft_Vote,Hard_Vote,label,Actual_Label
0,1,0.001269,0.998731,0.000938,0.999062,0.000179,0.999821,0.028954,0.971046,0.000074,0.999926,0.000250,0.999750,1,1,1,fake,fake
1,2,0.991674,0.008326,0.000884,0.999116,0.010019,0.989981,0.002718,0.997282,0.958628,0.041371,0.005722,0.994278,1,1,1,fake,fake
2,3,0.000385,0.999615,0.000958,0.999042,0.000084,0.999916,0.000053,0.999947,0.000048,0.999952,0.000249,0.999751,1,1,1,fake,fake
3,4,0.000416,0.999584,0.000936,0.999065,0.000109,0.999891,0.001268,0.998732,0.001172,0.998828,0.000251,0.999749,1,1,1,fake,fake
4,5,0.999851,0.000149,0.999052,0.000948,0.999784,0.000216,0.999951,0.000049,0.999972,0.000028,0.999373,0.000627,0,0,0,real,real


In [ ]:
temp = val_boosting_df[val_boosting_df["Actual_Label"]!=val_boosting_df["label"]].reset_index(drop=True)[["id", "label"]]
temp = temp.rename(columns={"label": "predicted_label"})
# temp

In [ ]:
wrong_df = temp.merge(val_df, on='id', how='inner')
wrong_df

,id,predicted_label,tweet,label
0,62,real,The Global #dietarysupplements market is antic...,fake
1,121,real,WHO reports record daily increase in global co...,fake
2,139,real,.@realdonaldtrump cherry-picked one stat to pr...,fake
3,146,real,There is no evidence that children have died b...,fake
4,149,real,The 2005 study found that chloroquine was effe...,fake
5,439,real,The N95 respirator offers the most protection ...,fake
6,454,real,An article from April announcing a new study “...,fake
7,497,real,In objection to WA's border closures @CliveFPa...,fake
8,610,fake,How deadly is the coronavirus #COVID19? Scient...,real
9,735,real,Health officials caution against the use of ch...,fake


In [ ]:
wrong_df.to_csv('/content/drive/MyDrive/Constraint/Submissions/wrong_results.csv', index=False)

Submission_v1: Robert+XLM-Robert+XLNET

Submission_v2: Robert+XLM-Robert+XLNET+DeBERT

Submission_v3: Robert+XLM-Robert+XLNET+ERNIE+DeBERTa

Submission_v4: Robert+XLM-Robert+XLNET+ERNIE+Electra

Submission_v5: ERNIE
